# Granite Route and Solve Agent

In this recipe, you will use the IBM [Granite](https://www.ibm.com/granite) model now available on watsonx.ai to build a Route and Solve Agent. This agent extends the [Function Calling Agent](../Function_Calling/Function_Calling_Agent.ipynb) recipe by introducing a router node that intelligently distributes queries to specialized subagents, each with their own grouped set of tools.

The Route and Solve architecture is ideal when you have a large number of tools that can be naturally grouped by category or domain. Instead of presenting all tools to a single agent, the router first determines which category of tools are needed, then routes the query to the appropriate subagent.

This approach offers several benefits:
- **Reduced tool selection errors**: Subagents only see relevant tools for their domain
- **Better scalability**: Easy to add new tool categories as subagents
- **Improved reasoning**: Specialized subagents can reason more effectively within their domain
- **Clear separation of concerns**: Tools are logically grouped by functionality


In [ ]:
from langgraph.prebuilt import ToolNode
from langchain_core.messages import AIMessage

# Define finance tools and subagent
finance_tools = [get_stock_price, get_stock_info]
finance_llm = llm.bind_tools(finance_tools)
print("Finance subagent created - Bound with finance tools: get_stock_price, get_stock_info")

# Define weather tools and subagent
weather_tools = [get_current_weather, get_weather_forecast]
weather_llm = llm.bind_tools(weather_tools)
print("Weather subagent created - Bound with weather tools: get_current_weather, get_weather_forecast")

# Finance subagent nodes
def finance_llm_node(state: State) -> State:
    messages = state["messages"]
    response_message = finance_llm.invoke(messages)
    return State(messages=[response_message], current_subagent="finance")

finance_tool_node = ToolNode(tools=finance_tools)
print("Finance subagent nodes configured - LLM and tool execution nodes ready")

# Weather subagent nodes
def weather_llm_node(state: State) -> State:
    messages = state["messages"]
    response_message = weather_llm.invoke(messages)
    return State(messages=[response_message], current_subagent="weather")

weather_tool_node = ToolNode(tools=weather_tools)
print("Weather subagent nodes configured - LLM and tool execution nodes ready")

# Steps

## Step 1. Set up your environment

While you can choose from several tools, this recipe is best suited for a Jupyter Notebook. Jupyter Notebooks are widely used within data science to combine code with various data sources such as text, images and data visualizations.

You can run this notebook in [Colab](https://colab.research.google.com/), or download it to your system and [run the notebook locally](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started_with_Jupyter_Locally/Getting_Started_with_Jupyter_Locally.md).

To avoid Python package dependency conflicts, we recommend setting up a [virtual environment](https://docs.python.org/3/library/venv.html).

Note, this notebook is compatible with Python 3.12 and well as Python 3.11, the default in Colab at the time of publishing this recipe. To check your python version, you can run the `!python --version` command in a code cell.

## Step 2. Set up a watsonx.ai instance

See [Getting Started with IBM watsonx](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started/Getting_Started_with_WatsonX.ipynb) for information on getting ready to use watsonx.ai.

You will need three credentials from the watsonx.ai set up to add to your environment: `WATSONX_URL`, `WATSONX_APIKEY`, and `WATSONX_PROJECT_ID`.

## Step 3. Install relevant libraries and set up credentials and the Granite model

We'll need a few libraries for this recipe. We will be using LangGraph and LangChain libraries to use Granite on watsonx.ai.

In [ ]:
! echo "::group::Install Dependencies"
%pip install uv
! uv pip install "git+https://github.com/ibm-granite-community/utils.git" \
    langgraph \
    langchain \
    langchain_ibm
! echo "::endgroup::"

Now we will get the credentials to use watsonx.ai and create the Granite model for use.

In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_core.utils.utils import convert_to_secret_str
from langchain.chat_models import init_chat_model

model = "ibm/granite-4-h-small"

llm_params = {
    "temperature": 0,
    "max_completion_tokens": 200,
    "repetition_penalty": 1.05,
}

# --- 1. LLM Initialization (Agent's Brain) ---
llm = init_chat_model(
    model=model,
    model_provider="ibm",
    url=convert_to_secret_str(get_env_var("WATSONX_URL")),
    apikey=convert_to_secret_str(get_env_var("WATSONX_APIKEY")),
    project_id=get_env_var("WATSONX_PROJECT_ID"),
    params=llm_params,
)
print(f"LLM initialized: {model}")



## Step 4: Define the functions grouped by category

In a Route and Solve agent, tools are organized into logical groups. Each group will be handled by a dedicated subagent. This organization allows the router to determine which category of tools is needed and route the query accordingly.

We'll create two categories: **Finance Tools** and **Weather Tools**. In a real-world scenario, you might have more categories such as Database Tools, Email Tools, or API Integration Tools.

### Finance Tools

In [ ]:
from langchain.tools import tool

@tool
def get_stock_price(ticker: str) -> str:
    """Fetches the current or historical stock price for a given ticker symbol (e.g., IBM)."""
    if ticker == "IBM":
        return "The current price for IBM is $180.50."
    elif ticker == "AAPL":
        return "The current price for AAPL is $235.80."
    elif ticker == "MSFT":
        return "The current price for MSFT is $442.30."
    return f"Stock price for {ticker} not found."

@tool
def get_stock_info(ticker: str) -> str:
    """Retrieves general information about a stock ticker including name, sector, and market cap."""
    stock_info = {
        "IBM": "IBM (International Business Machines): Technology sector, Market cap: $175B",
        "AAPL": "AAPL (Apple Inc.): Technology sector, Market cap: $2.9T",
        "MSFT": "MSFT (Microsoft): Technology sector, Market cap: $2.4T",
    }
    return stock_info.get(ticker.upper(), f"Information for {ticker} not found.")

print("=== Finance Tools Initialized ===")
print("  Tool 1: 'get_stock_price'")
print("  Tool 2: 'get_stock_info'")

### Weather Tools

In [ ]:
@tool
def get_current_weather(location: str) -> str:
    """Fetches the current weather for a given location (city name)."""
    weather_data = {
        "San Francisco": "San Francisco: Partly cloudy, 62°F, 65% humidity",
        "Boston": "Boston: Clear skies, 45°F, 55% humidity",
        "New York": "New York: Rainy, 50°F, 78% humidity",
    }
    location_title = location.title()
    return weather_data.get(location_title, f"Weather data for {location} not available.")

@tool
def get_weather_forecast(location: str, days: int = 5) -> str:
    """Provides the weather forecast for a specified city for the next N days (default: 5)."""
    forecasts = {
        "San Francisco": f"Next {days} days: Mix of sun and clouds with temps 55-65°F",
        "Boston": f"Next {days} days: Partly cloudy with temps 40-50°F",
        "New York": f"Next {days} days: Rain expected, temps 48-58°F",
    }
    location_title = location.title()
    return forecasts.get(location_title, f"Forecast for {location} not available.")

print("=== Weather Tools Initialized ===")
print("  Tool 1: 'get_current_weather'")
print("  Tool 2: 'get_weather_forecast'")

## Step 5: Build the Route and Solve Agent

Now we'll build the Route and Solve agent. This agent consists of:
1. A **router node** that analyzes the query and determines which category of tools to use
2. Multiple **subagent nodes**, each specialized for a specific tool category
3. A **loop mechanism** that allows the router to receive feedback and reroute if needed

### Step 5.1: Define the agent state

In [ ]:
from typing import Annotated, TypedDict, Literal
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    # The subagent that was routed to
    current_subagent: str

print("State schema defined - Tracks messages and routing decisions for graph flow.")

### Step 5.2: Create subagent graphs

We create separate function calling agents for each tool category. Each subagent only sees tools relevant to its domain.

In [ ]:
from langgraph.prebuilt import ToolNode
from langchain_core.messages import AIMessage

# Define finance tools and subagent
finance_tools = [get_stock_price, get_stock_info]
finance_llm = llm.bind_tools(finance_tools)
print("Finance subagent created - Bound with finance tools: get_stock_price, get_stock_info")

# Define weather tools and subagent
weather_tools = [get_current_weather, get_weather_forecast]
weather_llm = llm.bind_tools(weather_tools)
print("Weather subagent created - Bound with weather tools: get_current_weather, get_weather_forecast")

# Finance subagent nodes
def finance_llm_node(state: State) -> State:
    messages = state["messages"]
    response_message = finance_llm.invoke(messages)
    return State(messages=[response_message], current_subagent="finance")

finance_tool_node = ToolNode(tools=finance_tools)
print("Finance subagent nodes configured - LLM and tool execution nodes ready")

# Weather subagent nodes
def weather_llm_node(state: State) -> State:
    messages = state["messages"]
    response_message = weather_llm.invoke(messages)
    return State(messages=[response_message], current_subagent="weather")

weather_tool_node = ToolNode(tools=weather_tools)
print("Weather subagent nodes configured - LLM and tool execution nodes ready")

### Step 5.3: Create the router node

The router analyzes the user query and determines which subagent category should handle it.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

def router_node(state: State) -> State:
    """
    Router node that analyzes the query and routes to the appropriate subagent.
    
    Returns the state with a routing decision in the current_subagent field.
    The returned value should be 'finance', 'weather', or 'end' (for non-tool queries).
    """
    messages = state["messages"]
    user_query = messages[-1].content if messages else ""
    
    # Create a prompt for the router to classify the query
    router_prompt = f"""You are a router that directs queries to specialized agents.

Available agents:
- finance: Handles stock prices, market information, and financial queries
- weather: Handles weather forecasts, current weather, and climate information

Analyze the following query and respond with ONLY the agent name (finance, weather, or none).
If the query doesn't clearly fit any agent, respond with 'none'.

Query: {user_query}

Agent:"""
    
    # Call the model to make routing decision
    router_messages = [SystemMessage(content="You are a router. Respond with only the agent name: finance, weather, or none."),
                      HumanMessage(content=router_prompt)]
    response = llm.invoke(router_messages)
    routing_decision = response.content.strip().lower()
    
    # Normalize the response
    if "finance" in routing_decision:
        route = "finance"
    elif "weather" in routing_decision:
        route = "weather"
    else:
        route = "none"
    
    print(f"[Router]: Routing decision - {route}")
    return State(messages=messages, current_subagent=route)

print("Router node defined - Analyzes queries and routes to appropriate subagent")